In [7]:
import os, random, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import shutil
from scipy.stats import mode

In [2]:
def remove_row(d_set):
    tmp_d_set = d_set.drop_duplicates()
    if tmp_d_set[tmp_d_set.duplicated('WW')].shape[0] > 0:
        tmp_d_set = tmp_d_set.drop_duplicates('WW')#, keep='last')
    tmp_d_set = tmp_d_set.sort_values(by='WW')
    tmp_d_set = tmp_d_set.reset_index(drop=True)
    return tmp_d_set

In [9]:
if os.path.isdir('./data1/.ipynb_checkpoints'):
    shutil.rmtree('./data1/.ipynb_checkpoints')
f_list = os.listdir('./data1/')
f_list.sort()

product_file = [0, 8]
sale_file = [1,2,3,4,5,6,7]
inven_file = [9, 10, 11]

print(f_list)
print(inven_file)
print(product_file)
print(sale_file)


['02.Detail_생산계획 준수율.csv', 'Daimler_MktFcstWaterfall(Mkt Fcst Waterfall)_201901_202003.csv', 'Daimler_MktFcstWaterfall(Mkt Fcst Waterfall)_201909_202112.csv', 'Daimler_MktFcstWaterfall(Mkt Fcst Waterfall)_202004_202103.csv', 'Damiler_MktFcstWaterfall(Mkt Fcst Waterfall)_202009-202107.csv', 'Damiler_MktFcstWaterfall(Mkt Fcst Waterfall)_202103-202111.csv', 'Damiler_MktFcstWaterfall(Mkt Fcst Waterfall)_202103-202203.csv', 'Damiler_MktFcstWaterfall(Mkt Fcst Waterfall)_202107-202207.csv', 'DynamicResult.csv', 'Inven_3xxC.csv', 'Inven_3xxT.csv', 'Inven_5xx0.csv']
[9, 10, 11]
[0, 8]
[1, 2, 3, 4, 5, 6, 7]


## Inven

In [12]:
for i in inven_file:
    if '3xxC' in f_list[i]:
        tmp_3xxC = pd.read_csv('./data1/'+f_list[i])
        tmp_3xxC.columns = list(['Week', 'Date', 'Inven'])

tmp_inven = tmp_3xxC[['Week', 'Inven']]
tmp_ww = np.array(tmp_inven['Week'])
worknum = tmp_inven['Week']
tmp_inven_prev = []
for i in range(0, len(tmp_ww)):
    tmp_prev_inven = []
    for j in range(1,9):
        if i-j < 0 :
            tmp_prev_inven.append(-999)
        else:
            tmp_prev_inven.append(tmp_inven.iloc[(i-j),1]) #(tmp_inven.iloc[(i-j),4])
    tmp_inven_prev.append(tmp_prev_inven)
tmp_inven_prev = pd.DataFrame(tmp_inven_prev)
inven_prev_name = list(map(lambda x : 'Prev_Inven_'+str(x), np.arange(1, 9, 1)))
tmp_inven_prev.columns = inven_prev_name
tmp_inven = pd.concat([tmp_inven, tmp_inven_prev], axis=1)
tmp_inven.to_csv('E61D_Inven.csv', index=False)

## Product

In [13]:
for m in product_file:
    tmp_product = pd.read_csv('./data1/'+f_list[m])       
    if 'Dynamic' in f_list[m]:      
        plan_ww_unique = tmp_product['Plan Date'].unique()
        plan_ww_unique.sort()
        target_ww_unique = tmp_product['Date'].unique()
        target_ww_unique.sort()

        for i in range(0, len(plan_ww_unique)):
            tmp_post = []
            for j in range(1, 13):
                if i+j < len(plan_ww_unique) :
                    if tmp_product[(tmp_product['Plan Date']==plan_ww_unique[i]) & (tmp_product['Date']==(plan_ww_unique[i+j]))].shape[0] > 0:
                        tmp_post_val = tmp_product[(tmp_product['Plan Date']==plan_ww_unique[i]) &(tmp_product['Date']==(plan_ww_unique[i+j]))].iloc[0,2]
                        tmp_post.append(tmp_post_val)
                    else:
                        tmp_post.append(-999)
                else:
                    tmp_post.append(-999)                    
            if i == 0:
                tmp_product_post = tmp_post
            else:
                tmp_product_post = np.vstack((tmp_product_post, tmp_post))
        product_post = pd.DataFrame(tmp_product_post)
        product_post_name = list(map(lambda x : 'Post_Product_'+str(x), np.arange(1, 13, 1)))
        product_post.columns = product_post_name        
        product_post['WW'] = plan_ww_unique

        plan_ww_unique = tmp_product['Plan Date'].unique()
        plan_ww_unique.sort()
        target_ww_unique = tmp_product['Date'].unique()
        target_ww_unique.sort()

        for i in range(0, len(target_ww_unique)):
            tmp_var = []
            for j in range(1, 13):
                index_col = i-j if i-j > 0 else 0

                if tmp_product[(tmp_product['Date']==target_ww_unique[i]) & (tmp_product['Plan Date']==(target_ww_unique[index_col]))].shape[0] > 0:
                    tmp_var_val = tmp_product[(tmp_product['Date']==target_ww_unique[i]) &(tmp_product['Plan Date']==(target_ww_unique[index_col]))].iloc[0,2]
                    tmp_var.append(tmp_var_val)
                else:
                    tmp_var.append(-999)
            if i == 0:
                tmp_product_var = tmp_var
            else:
                tmp_product_var = np.vstack((tmp_product_var, tmp_var))

        product_var = pd.DataFrame(tmp_product_var)
        product_var_name  = list(map(lambda x : 'Var_Product_'+str(x),  np.arange(1, 13, 1)))
        product_var.columns = product_var_name
        product_var['WW'] = target_ww_unique    
    else:          
        tmp_product = tmp_product.sort_values(by=['Plan Week'], axis=0)
        product_set = tmp_product[['Plan Week', 'Result Qty_x']].copy()        
        product_set.columns = list(['Week', 'Product'])

tmp_product_ww = tmp_inven['Week'] # product_set['Week']
for i in range(0, len(tmp_product_ww)):
    tmp_prev = []
    for j in range(1,13):
        if product_set[product_set['Week'] == (tmp_product_ww[i]-j)].shape[0]>0:
            tmp_prev_val = product_set[product_set['Week'] == (tmp_product_ww[i]-j)].iloc[0,1]
            tmp_prev.append(tmp_prev_val)
        else:
            tmp_prev.append(-999)
    if i == 0:
        product_prev = tmp_prev
    else:
        product_prev = np.vstack((product_prev, tmp_prev))
product_prev = pd.DataFrame(product_prev)
product_prev_name  = list(map(lambda x : 'Prev_Product_'+str(x),  np.arange(1, 13, 1)))
product_prev.columns = product_prev_name        
product_prev['WW'] = tmp_product_ww

product = pd.merge(pd.merge(pd.merge(pd.merge(worknum, product_set, how='left', on='Week'), 
                                    product_var, how='left', left_on='Week', right_on='WW'), 
                    product_post, how='left', left_on='Week', right_on='WW'), 
        product_prev, how='left', left_on='Week', right_on='WW')
ww_col = [z for i,z in enumerate(product.columns) if 'WW' in z]
product = product.drop(columns = ww_col)
product.to_csv('E61D_Product.csv', index=False)

KeyError: 'Plan Week'